In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=cacfbf6e248e740533c0304bacb240934819cea9fdbda6e93202887eb762c56c
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import *
spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_2")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

##Данные: google drive: raw_sales.csv
###Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):

####date of sale
####price
####property type
####number of bedrooms
####4digit postcode

In [3]:
data = spark.read.csv('raw_sales.csv', header=True)
data.show(5)

+-------------------+--------+------+------------+--------+
|           datesold|postcode| price|propertyType|bedrooms|
+-------------------+--------+------+------------+--------+
|2007-02-07 00:00:00|    2607|525000|       house|       4|
|2007-02-27 00:00:00|    2906|290000|       house|       3|
|2007-03-07 00:00:00|    2905|328000|       house|       3|
|2007-03-09 00:00:00|    2905|380000|       house|       4|
|2007-03-21 00:00:00|    2906|310000|       house|       3|
+-------------------+--------+------+------------+--------+
only showing top 5 rows



##Задание 1
###Добавьте к таблице следующие поля:

###Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode)
###Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)
###Стоимость последнего проданного дома до текущего

In [4]:
from pyspark.sql import Window
#from pyspark.sql.pandas.functions import pandas_udf, PandasUDFType
w = Window.partitionBy("postcode").orderBy("datesold")

data_new = data.withColumn("average_cost_10_before", F.avg(F.col('price')).over(w.rowsBetween(Window.currentRow-10, Window.currentRow-1)))\
.withColumn("average_cost_10_after", F.avg(F.col('price')).over(w.rowsBetween(Window.currentRow+1, Window.currentRow+10)))\
.withColumn("cost_last_before", F.lag(F.col('price'), 1).over(w))
         

In [5]:
data_new.show(5)

+-------------------+--------+------+------------+--------+----------------------+---------------------+----------------+
|           datesold|postcode| price|propertyType|bedrooms|average_cost_10_before|average_cost_10_after|cost_last_before|
+-------------------+--------+------+------------+--------+----------------------+---------------------+----------------+
|2007-07-08 00:00:00|    2600|327000|       house|       1|                  null|             708350.0|            null|
|2007-08-16 00:00:00|    2600|790000|       house|       4|              327000.0|             698350.0|          327000|
|2007-12-05 00:00:00|    2600|825000|       house|       3|              558500.0|             679350.0|          790000|
|2008-01-21 00:00:00|    2600|315000|        unit|       1|     647333.3333333334|             742850.0|          825000|
|2008-04-24 00:00:00|    2600|292500|       house|       1|              564250.0|             786600.0|          315000|
+-------------------+---

##Задание 2
###В итоге у вас таблица с колонками (или нечто похожее):

###price
###Среднегодовая цена
###Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
###Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
###Стоимость последнего проданного дома до текущего ((1 балл)
и др.
###Посчитайте кол-во уникальных значений в каждой строчке (unique(row)) (ипользуйте udf). Попробуйте сделать то же самое используя pandas udf.

In [6]:
data_new.withColumn('array_distinct', F.size(F.array_distinct(F.array(data_new.columns)))).show()

+-------------------+--------+-------+------------+--------+----------------------+---------------------+----------------+--------------+
|           datesold|postcode|  price|propertyType|bedrooms|average_cost_10_before|average_cost_10_after|cost_last_before|array_distinct|
+-------------------+--------+-------+------------+--------+----------------------+---------------------+----------------+--------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                  null|             708350.0|            null|             7|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|              327000.0|             698350.0|          327000|             8|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|              558500.0|             679350.0|          790000|             8|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|     647333.3333333334|             742850.0|          825000|             8|
|2008-04-24 00:00:00|    2600| 292

In [7]:
data_new.distinct() 

DataFrame[datesold: string, postcode: string, price: string, propertyType: string, bedrooms: string, average_cost_10_before: double, average_cost_10_after: double, cost_last_before: string]

##Задание 3
###SQL like case when или if elif else

###Создайте колонку,в которой будет отображаться "+", "-" или "=", если "Средняя стомость 10 проданных домов до текущего в том же районе" больше, меньше или равно "Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)", соотвественно.

###Если одно из полей Null, запишите в эту колонку "Нет данных"

In [8]:
data_new.withColumn('+, -, =', F.when(F.col('average_cost_10_before') > F.col('average_cost_10_after'), '+')\
                    .when(F.col('average_cost_10_before') < F.col('average_cost_10_after'), '-')\
                    .when(F.col('average_cost_10_before').isNull() | F.col('average_cost_10_after').isNull(), 'Нет данных')\
                    .otherwise('=')
).show()

+-------------------+--------+-------+------------+--------+----------------------+---------------------+----------------+----------+
|           datesold|postcode|  price|propertyType|bedrooms|average_cost_10_before|average_cost_10_after|cost_last_before|   +, -, =|
+-------------------+--------+-------+------------+--------+----------------------+---------------------+----------------+----------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|                  null|             708350.0|            null|Нет данных|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|              327000.0|             698350.0|          327000|         -|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|              558500.0|             679350.0|          790000|         -|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|     647333.3333333334|             742850.0|          825000|         -|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|  